# Using the module to solve Colley's system


## Script to compile the module


In [11]:
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python3)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 9.3.0
-- The CXX compiler identification is GNU 9.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- pybind11 v2.6.1 
-- Found PythonInterp: /usr/bin/python3 (found version "3.8.5") 
-- Found PythonLibs: /usr/lib/x86_64-linux-gnu/libpython3.8.so
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- Configuring done
-- Generating done
-- Build files have been written to: /home/lisandro/Documents/Dev/Cpp/ColleyMatrix/build

In [1]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python3 --version
import Solver


/home/lisandro/Documents/Dev/Cpp/ColleyMatrix/notebooks
Python 3.8.5


In [8]:
import numpy as np
import pandas as pd

# Simple example

In [9]:
import sys
sys.path.insert(1, "../ColleyTransform/")
import ColleyTransform 

In [10]:
df = pd.DataFrame(data={
    "home_team" : [1,2,2,3,3,4,5,5,6,6,4,3,3],
    "home_team_score" : [16,38,0,34,23,31,0,38,1,1,1,1,1],
    "away_team": [4,5,6,1,4,1,6,4,2,5,6,1,2],
    "away_team_score" : [13,17,1,21,10,6,1,23,0,0,0,0,0]
})


In [11]:
d, r, b = ColleyTransform.transformToColley(df, ["home_team","home_team_score"], ["away_team","away_team_score"], "")

In [64]:
S = Solver.Solver(r)
S.fit()

In [65]:
ss = (S.getLMatrix() @ S.getLMatrix().T)

In [66]:
# Check cholesky worked doing A =? L . Lt
np.allclose(A, ss)

True

In [67]:
S.predict(b)

array([0.40412234, 0.39880319, 0.77420213, 0.43816489, 0.35079787,
       0.63390957])

# NBA 2016 Scores

In [3]:
nba = pd.read_csv("nba_2016_scores.csv")

In [5]:
nba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1002 entries, 0 to 1001
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   id               1002 non-null   int64
 1   date             1002 non-null   int64
 2   home_team        1002 non-null   int64
 3   home_outcome     1002 non-null   int64
 4   home_team_score  1002 non-null   int64
 5   away_team        1002 non-null   int64
 6   away_outcome     1002 non-null   int64
 7   away_team_score  1002 non-null   int64
dtypes: int64(8)
memory usage: 62.8 KB


In [12]:
d, r, b = ColleyTransform.transformToColley(nba, ["home_team","home_team_score"], ["away_team","away_team_score"], "")

In [16]:
S = Solver.Solver(r)
S.fit()

In [17]:
results = S.predict(b)

In [20]:
d.keys()

dict_keys([9, 5, 10, 30, 2, 16, 28, 8, 20, 21, 6, 7, 25, 13, 18, 15, 1, 29, 27, 26, 24, 17, 11, 4, 12, 22, 14, 19, 3, 23])

In [32]:
prediction = pd.DataFrame(data = {
    "id" : list(d.keys()),
    "ratings" : results
})

In [35]:
teams_labels = pd.read_csv("nba_2016_teams.csv")

prediction = pd.merge(prediction, teams_labels, on="id")


In [41]:
prediction = prediction.sort_values(by="ratings", ascending=False)

In [43]:
prediction

,id,ratings,team
2,10,0.874063,Golden_State
18,27,0.803976,San_Antonio
10,6,0.692428,Cleveland
6,28,0.659347,Toronto
9,21,0.655552,Oklahoma_City
13,13,0.629708,LA_Clippers
5,16,0.577367,Miami
4,2,0.565690,Boston
15,15,0.563176,Memphis
16,1,0.558822,Atlanta


### Comparing the results

The official result of the 2015-2016 season was the next one:

![results](misc.png)

Comparing what we get using Colley's method, we can se a great prediction!